In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [50]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout=0.0, learning_rate=0.001, num_epochs=20):
        super(GRU, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        
        # GRU слой
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        
        # Полносвязный слой для предсказания выхода
        self.fc = nn.Linear(hidden_size, output_size)

        # Функция потерь и оптимизатор
        self.criterion = nn.MSELoss()
        self.optimizer = None

    def forward(self, x):
        # Ожидаем входной размер (batch_size, input_size)
        x = x.unsqueeze(1)  # Добавляем фиктивный размер для sequence_length        

        # Инициализация скрытых состояний (h0)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Прямой проход через GRU        
        out, _ = self.gru(x, h0)
        
        # Используем только последнее скрытое состояние
        out = out[:, -1, :]
        
        # Прогон через полносвязный слой
        out = self.fc(out)        
        
        return out

    def train_model(self, dataloader, device, verbosity: int = 1):
        self.to(device)

        # Инициализация оптимизатора
        self.optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)

        for epoch in range(self.num_epochs):
            self.train()

            for x_batch, y_batch in dataloader:                
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)

                # Прямой проход
                outputs = self(x_batch)
                loss = self.criterion(outputs, y_batch)

                # Обратное распространение и обновление весов
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

            # Печать информации об обучении
            if (epoch + 1) % verbosity == 0:
                print(f'Epoch [{epoch + 1}/{self.num_epochs}], Loss: {loss.item():.4f}')

        print("Обучение завершено!")

    def predict(self, dataloader, device):
        self.eval()
        predictions = []
        with torch.inference_mode():
            for x_batch, _ in dataloader:
                x_batch = x_batch.to(device)
                outputs = self(x_batch)
                predictions.append(outputs.cpu())
        return torch.cat(predictions, dim=0)

In [51]:
# 1/n sequence generator
def one_by_n_generator(n):    
    for i in range(n):
        yield 1 / (i + 1)

In [52]:
# Arithmetic progression
def arithmetic_progression(n, a0, d):
    for i in range(n):
        yield a0 + i * d

In [53]:
# 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,... sequence generator
def one_zero_generator(n):
    count = 1
    generated = 0
    while generated < n:
        generated += 1
        yield 1
        for _ in range(count):
            if generated >= n:
                break
            generated += 1
            yield 0            
        count += 1

In [54]:
# Prepare data using sliding window
def create_sliding_window_data(sequence, window_size, output_size):
    X, y = [], []
    
    # Формирование данных скользящего окна
    for i in range(len(sequence) - window_size - output_size + 1):
        X.append(sequence[i:i + window_size])
        y.append(sequence[i + window_size:i + window_size + output_size])
    
    # Преобразование в массивы numpy
    X, y = np.array(X), np.array(y)

    return X, y

In [55]:
class SequenceDataset(Dataset):

    def __init__(self, x, y):
        super(SequenceDataset, self).__init__()

        self.x = torch.as_tensor(x, dtype=torch.float32)
        self.y = torch.as_tensor(y, dtype=torch.float32)

        self.n_samples = self.x.shape[0]
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_samples

In [56]:
# Гиперпараметры
sequence_length = 10
window_size = 3
hidden_size = 4     # Размер скрытого состояния
output_size = 2      # Размер выхода (например, регрессия)
num_layers = 2       # Количество слоёв GRU
learning_rate = 0.001
num_epochs = 10000
dropout = 0.2  # Dropout для регуляризации

# Создание модели
model = GRU(window_size, hidden_size, output_size, num_layers, dropout, learning_rate, num_epochs)

# Пример генерации данных вида 1 / n (для обучения)
batch_size = 16
sequence = list(arithmetic_progression(sequence_length, 1, 1))
x, y = create_sliding_window_data(sequence, window_size, output_size)
x_train, y_train = x[0:int(len(x) * 0.8)], y[0:int(len(x) * 0.8)]
x_test, y_test = x[int(len(x) * 0.8):], y[int(len(x) * 0.8):]

train_dataloader = DataLoader(dataset=SequenceDataset(x_train, y_train),
                              batch_size=batch_size,
                              shuffle=False)

# Перенос на устройство (GPU, если доступно)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Тренировка модели
model.train_model(train_dataloader, device, 1000)

Epoch [1000/10000], Loss: 3.9037
Epoch [2000/10000], Loss: 1.4004
Epoch [3000/10000], Loss: 0.2754
Epoch [4000/10000], Loss: 0.0720
Epoch [5000/10000], Loss: 0.0363
Epoch [6000/10000], Loss: 0.3102
Epoch [7000/10000], Loss: 0.0151
Epoch [8000/10000], Loss: 0.0384
Epoch [9000/10000], Loss: 0.0994
Epoch [10000/10000], Loss: 0.0307
Обучение завершено!


In [57]:
# Пример данных для предсказания
test_dataloader = DataLoader(dataset=SequenceDataset(x_test, y_test),
                             batch_size=batch_size,
                             shuffle=False)
predictions = model.predict(test_dataloader, device)
mae = np.mean(np.absolute(y_test - predictions.numpy()))
print(f'Test data MAE: {mae:.4f}%')
# Mean absolute percentage error
mape = np.mean(np.absolute((y_test - predictions.numpy()) / y_test)) * 100
print(f'Test data MAPE: {mape:.4f}%')

Test data MAE: 1.2456%
Test data MAPE: 13.6507%


In [58]:
predictions

tensor([[7.1976, 8.2066],
        [7.2993, 8.3140]])

In [59]:
y_test

array([[ 8,  9],
       [ 9, 10]])